# Read the .atm files, show it in the graph and add to the database

import librares


In [22]:
import os
from turtle import color
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from turtle import color
import seaborn as sns
from sklearn.linear_model import LinearRegression 
from pathlib import Path
import plotly.express as px

Add the name of the spectr and additional information

In [23]:
ats_sp_name = 'D:\work\\1315\Коэф перехода 5174\Sr-90\SrM10.5174.ats'
Name = ats_sp_name[ats_sp_name.rindex('\\')+1:-4]
coments = 'методика поверки бета'
nuclid = 'Sr-90'
Name

'SrM10.5174'

Handling the spectr and check the data

In [24]:
E_cal = []
R_cal = []
Rest = []
sp_gamma = []
dict_d_ats ={}
dict_d_ats['name'] = Name
dict_d_ats['nuclid'] = nuclid
with open (ats_sp_name ) as spec:
    n_line = 0
    for line in spec:
        if line.startswith('TIME'): time = int(line[line.index('=')+2 :-1])
        elif line.startswith('GEOMETRY'): geom = line[line.index('=')+2 :-1]
        elif line.startswith('WEIGHT'): weight = float(line[line.index('=')+2 :-1])
        elif line.startswith('DATE '): date = line[line.index('=')+2 :-1]
        elif line.startswith('DU_TYPE'): s_n = line[line.index('=')+2 :-1]
        
        n_line += 1
        if n_line in range(48,1071): 
            E_cal.append(float(line.strip()))
        if n_line in range(2098,3121): 
            R_cal.append(float(line.strip()))
        if n_line in range(3123,4145): 
            sp_gamma.append(float(line.strip()))
        if n_line in range(1073,2096): 
            Rest.append(float(line.strip()))
sp_gamma.append(0)
dict_d_ats = {'time':time,
             'serial_number':s_n, 
             'geometry':geom, 
             'weight':weight, 
             'date':date, 
             'sp_beta':[Rest], 
             'Energy':[E_cal], 
             'sp_gamma':[sp_gamma], 
             'R_cal':[R_cal]
             }
def check_data(data_dict):
    col = ['sp_gamma','sp_beta', 'Energy', 'R_cal']
    d = [data_dict[i] for i in col]
    l = [len(i) for i in d]
    return len(set(l)) == 1 
check_data(dict_d_ats)


dict_d_ats['coments'] = coments
dict_d_ats['nuclid'] = nuclid

compose the dataframe

In [25]:
df_ats = pd.DataFrame(dict_d_ats)


show spectr in the graph

In [26]:
def show_spectr(data):
    fig = px.scatter(data['sp_gamma'][0])
    fig1 = px.scatter(data['sp_beta'][0])
    fig.show()
    fig1.show()

# show_spectr(df_ats)

add spectr to the database


In [27]:
df_ats

,time,serial_number,geometry,weight,date,sp_beta,Energy,sp_gamma,R_cal,coments,nuclid
0,4200,5174,0,1002.0,09/15/2021 06:09:21,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 4.0, 7.0, 10.1, 13.1, 16.20000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.1, 1.2, 1.2, 1.2, 1.2, 1.3, 1.3, 1.3, 1.3, ...",методика поверки бета,Sr-90


In [30]:
db = pd.read_csv('1315_database.csv')
db.columns= ['name', 'nuclid', 'time', 'serial_number',	'geometry',	'weight', 'date', 'sp_beta', 'Energy', 'sp_gamma',	'R_cal', 'coments' ]

if df_ats['name'][0] in db['name'].values:
    db[db['name'] == Name] = df_ats.values
else:
    db = pd.concat([db, df_ats], ignore_index=True)
db.to_csv('1315_database.csv', index=False)
db.name

KeyError: 'name'